# Função para receber o gênero de uma música do dataset de acordo com seu ID

In [44]:
!rm -r /content/musicas.zip

In [1]:
!unzip /content/espectrogramas.zip

Archive:  /content/espectrogramas.zip
   creating: espectrogramas/
  inflating: espectrogramas/1.png    
  inflating: espectrogramas/10.png   
  inflating: espectrogramas/100.png  
  inflating: espectrogramas/101.png  
  inflating: espectrogramas/102.png  
  inflating: espectrogramas/103.png  
  inflating: espectrogramas/104.png  
  inflating: espectrogramas/105.png  
  inflating: espectrogramas/106.png  
  inflating: espectrogramas/107.png  
  inflating: espectrogramas/108.png  
  inflating: espectrogramas/109.png  
  inflating: espectrogramas/11.png   
  inflating: espectrogramas/110.png  
  inflating: espectrogramas/111.png  
  inflating: espectrogramas/112.png  
  inflating: espectrogramas/113.png  
  inflating: espectrogramas/114.png  
  inflating: espectrogramas/115.png  
  inflating: espectrogramas/116.png  
  inflating: espectrogramas/117.png  
  inflating: espectrogramas/118.png  
  inflating: espectrogramas/119.png  
  inflating: espectrogramas/12.png   
  inflating: espectro

In [2]:
import pandas as pd

def check_gender_by_id(filename, id) -> None | str:
    df = pd.read_excel(filename)

    genero = df.loc[df['id'] == id, 'genero']
    return genero.iloc[0]

In [3]:
# Exemplo de uso
arquivo = '/content/generos.xlsx'
id_musica = 17  # Coloque o ID desejado
genero = check_gender_by_id(arquivo, id_musica)
print(f'O gênero da música com ID {id_musica} é: {genero}')

O gênero da música com ID 17 é: Pop


In [4]:
def dividir_musicas_em_listas(arquivo_xlsx):
    # Carrega o arquivo Excel para um DataFrame
    df = pd.read_excel(arquivo_xlsx)

    # Dicionário para armazenar as músicas de cada gênero
    generos = {}

    # Itera por todas as músicas no arquivo
    for id_musica in df['id']:
        # Usa a função check_gender_by_id para obter o gênero da música
        genero = check_gender_by_id(arquivo_xlsx, id_musica)

        # Se o gênero for válido (não None e dentro da lista de gêneros válidos), adiciona o ID
        if genero not in generos:
            generos[genero] = []
        generos[genero].append(id_musica)

    # Listas para armazenar as músicas de treino, validação e teste
    musicas_treino = []
    musicas_validacao = []
    musicas_teste = []

    # Agora, vamos dividir as músicas por gênero em treino, validação e teste de forma aleatória
    for genero, musicas in generos.items():
        # Embaralha as músicas para aleatoriedade
        random.shuffle(musicas)

        # Verifica se há pelo menos 30 músicas para dividir

        if len(musicas)<10:
          print(f"Erro: genero {genero} com poucas músicas")
          return
        else:
          total = len(musicas)

          idx_70 = int(0.7 * total) #limites para dividir com qualquer tamanho
          idx_90 = int(0.9 * total)

          treino = musicas[:idx_70]
          validacao = musicas[idx_70:idx_90]
          teste = musicas[idx_90:]

          # Adiciona as listas para cada categoria
          musicas_treino.extend(treino)
          musicas_validacao.extend(validacao)
          musicas_teste.extend(teste)


    # Retorna as 3 listas
    return musicas_treino, musicas_validacao, musicas_teste

In [5]:
import random
treino,validacao,teste = dividir_musicas_em_listas('/content/generos.xlsx')

print(teste)

[221, 194, 255, 220, 234, 268, 189, 9, 243, 193, 187, 77, 274, 98, 360, 364, 308, 296, 373, 310, 330, 288, 297, 378, 293, 309, 26, 101, 29, 460, 466, 428, 435, 406, 419, 474, 384, 430, 480, 461, 458, 36, 549, 538, 507, 557, 569, 515, 526, 500, 489, 540, 510, 560, 142, 597, 56, 165, 57, 679, 641, 615, 652, 654, 635, 594, 610, 632]


In [6]:
for x in teste:
  print(check_gender_by_id('/content/generos.xlsx',x))

Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Eletronical
Eletronical
Eletronical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country


In [7]:
# Exemplo: Gêneros associados aos IDs
id_to_genre = {
    0: "Rock",
    1: "Pop",
    2: "Jazz",
    3: "Eletronical",
    4: "Classical",
    5: "Country"
    # Adicione todos os IDs e seus gêneros
}

# Converta os gêneros para índices numéricos para o treinamento
genre_to_index = {genre: idx for idx, genre in enumerate(set(id_to_genre.values()))}

# Exemplo: rock -> 0, pop -> 1, jazz -> 2
id_to_label = {id: genre_to_index[genre] for id, genre in id_to_genre.items()}

print(id_to_genre)
print(genre_to_index)
print(id_to_label)


{0: 'Rock', 1: 'Pop', 2: 'Jazz', 3: 'Eletronical', 4: 'Classical', 5: 'Country'}
{'Country': 0, 'Eletronical': 1, 'Jazz': 2, 'Pop': 3, 'Rock': 4, 'Classical': 5}
{0: 4, 1: 3, 2: 2, 3: 1, 4: 5, 5: 0}


In [8]:
import tensorflow as tf
import os


base_dir = "/content/espectrogramas"

def id_to_filepath(id):
  return os.path.join(base_dir,f"{id}.png")

In [9]:
df = pd.read_excel('/content/generos.xlsx')

# Cria o mapeamento de gêneros para índices
genre_to_index = {genre: idx for idx, genre in enumerate(set(df['genero']))}

# Mapeia IDs das músicas aos índices das classes
id_to_label = {row['id']: genre_to_index[row['genero']] for _, row in df.iterrows()}

print("id_label:", id_to_label)
print(genre_to_index)

id_label: {1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 4, 10: 4, 11: 3, 12: 3, 13: 3, 14: 3, 15: 3, 16: 3, 17: 3, 18: 3, 19: 3, 20: 3, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 5, 32: 5, 33: 5, 34: 5, 35: 5, 36: 5, 37: 5, 38: 5, 39: 5, 40: 5, 41: 2, 42: 2, 43: 2, 44: 2, 45: 2, 46: 2, 47: 2, 48: 2, 49: 2, 50: 2, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 4, 62: 4, 63: 4, 64: 4, 65: 4, 66: 4, 67: 4, 68: 4, 69: 4, 70: 4, 71: 4, 72: 4, 73: 4, 74: 4, 75: 4, 76: 4, 77: 4, 78: 4, 79: 4, 80: 4, 81: 3, 82: 3, 83: 3, 84: 3, 85: 3, 86: 3, 87: 3, 88: 3, 89: 3, 90: 3, 91: 3, 92: 3, 93: 3, 94: 3, 95: 3, 96: 3, 97: 3, 98: 3, 99: 3, 100: 3, 101: 1, 102: 1, 103: 1, 104: 1, 105: 1, 106: 1, 107: 1, 108: 1, 109: 1, 110: 1, 111: 1, 112: 1, 113: 1, 114: 1, 115: 1, 116: 1, 117: 1, 118: 1, 119: 1, 120: 1, 121: 5, 122: 5, 123: 5, 124: 5, 125: 5, 126: 5, 127: 5, 128: 5, 129: 5, 130: 5, 131: 5, 132: 5, 133: 5, 134: 5, 135: 5, 136: 5, 137: 5, 1

In [10]:
def create_tf_dataset_with_labels(id_list, id_to_label):
    filepaths = [id_to_filepath(id) for id in id_list]
    labels = [id_to_label[id] for id in id_list]

    # Cria um dataset contendo pares (caminho, rótulo)
    dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))

    # Carrega a imagem e retorna (imagem, rótulo)
    def load_image_and_label(filepath, label):
        image = tf.io.read_file(filepath)
        image = tf.image.decode_png(image, channels=3)
        image = tf.image.resize(image, [224, 224])  # Tamanho esperado pela CNN
        return image, label

    dataset = dataset.map(load_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset


In [11]:
def normalizar_imagem(imagem):
    return tf.cast(imagem, tf.float32) / 255.0  # Normaliza para o intervalo [0, 1]

train_dataset = create_tf_dataset_with_labels(treino, id_to_label)
val_dataset = create_tf_dataset_with_labels(validacao, id_to_label)
test_dataset = create_tf_dataset_with_labels(teste, id_to_label)

train_dataset = train_dataset.map(lambda imagem, label: (normalizar_imagem(imagem), label))
val_dataset = val_dataset.map(lambda imagem, label: (normalizar_imagem(imagem), label))
test_dataset = test_dataset.map(lambda imagem, label: (normalizar_imagem(imagem), label))


# Agrupar em lotes e otimizar
batch_size = 32
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [12]:
import cv2 as cv

img = cv.imread("/content/espectrogramas/125.png")


input_shape = img.shape
print(input_shape)
num_classes = len(set(id_to_label.values()))
print(f"Quantidade de classes: {num_classes}")

print(num_classes)

(924, 2325, 3)
Quantidade de classes: 6
6


In [13]:
import os
from PIL import Image

def verificar_tamanhos_canais_pasta(pasta):
    tamanhos = {}
    canais = {}
    arquivos_verificados = 0

    for root, _, files in os.walk(pasta):
        for file in files:
            if file.endswith((".png", ".jpg", ".jpeg")):  # Ajuste as extensões conforme necessário
                caminho_completo = os.path.join(root, file)
                try:
                    with Image.open(caminho_completo) as img:
                        tamanho = img.size  # (largura, altura)
                        modo = img.mode  # Ex.: "L", "RGB", "RGBA"
                        canais_count = {'L': 1, 'RGB': 3, 'RGBA': 4}.get(modo, "Desconhecido")

                        tamanhos[caminho_completo] = tamanho
                        canais[caminho_completo] = canais_count
                        arquivos_verificados += 1
                except Exception as e:
                    print(f"Erro ao processar {caminho_completo}: {e}")

    if arquivos_verificados == 0:
        print("Nenhum arquivo de imagem foi encontrado na pasta especificada.")
        return

    # Identificar se todos têm o mesmo tamanho e canais
    tamanhos_unicos = set(tamanhos.values())
    canais_unicos = set(canais.values())

    print(f"\nTotal de imagens verificadas: {arquivos_verificados}")
    if len(tamanhos_unicos) == 1:
        print(f"Todos os espectrogramas têm o mesmo tamanho: {tamanhos_unicos.pop()}")
    else:
        print("Os tamanhos dos espectrogramas variam. Detalhes:")
        for caminho, tamanho in tamanhos.items():
            print(f"{caminho}: {tamanho}")

    if len(canais_unicos) == 1:
        print(f"Todas as imagens têm o mesmo número de canais: {canais_unicos.pop()}")
    else:
        print("As imagens têm diferentes números de canais. Detalhes:")
        for caminho, num_canais in canais.items():
            print(f"{caminho}: {num_canais} canais")

# Substitua pelo caminho da sua pasta de espectrogramas
caminho_pasta = "/content/espectrogramas"
verificar_tamanhos_canais_pasta(caminho_pasta)


Total de imagens verificadas: 679
Todos os espectrogramas têm o mesmo tamanho: (2325, 924)
Todas as imagens têm o mesmo número de canais: 4


In [46]:
from tensorflow.keras.optimizers import Adam

# Defina o learning rate desejado
learning_rate = 0.0005  # Por exemplo, 0.001

# Crie uma instância do otimizador Adam com o learning rate ajustado
optimizer = Adam(learning_rate=learning_rate)

In [47]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3, activation='relu', input_shape=(2325,924,3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))
cnn.add(tf.keras.layers.GlobalAveragePooling2D())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
cnn.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

cnn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 2323, 922, 32)       │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 1161, 461, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 1159, 459, 64)       │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 579, 229, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 577, 227, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 288, 113, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 110,534 (431.77 KB)

 Trainable params: 110,534 (431.77 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
history = cnn.fit(train_dataset, epochs=150, validation_data=val_dataset)

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 20s 948ms/step - accuracy: 0.2461 - loss: 1.8210 - val_accuracy: 0.1912 - val_loss: 1.7779
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 886ms/step - accuracy: 0.4327 - loss: 1.7086 - val_accuracy: 0.1912 - val_loss: 1.7699
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 22s 950ms/step - accuracy: 0.3651 - loss: 1.7381 - val_accuracy: 0.1912 - val_loss: 1.7619
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 13s 880ms/step - accuracy: 0.3488 - loss: 1.7332 - val_accuracy: 0.2132 - val_loss: 1.7497
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 14s 946ms/step - accuracy: 0.3339 - loss: 1.7186 - val_accuracy: 0.2794 - val_loss: 1.7370
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 20s 872ms/step - accuracy: 0.2607 - loss: 1.7189 - val_accuracy: 0.3162 - val_loss: 1.7291
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.2857 - loss: 1.7076 - val_accuracy: 0.3162 - val_loss: 1.7218
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 13s 889ms/step - accuracy: 0.2694 - loss: 1.7096 - val

In [49]:
loss, accuracy = cnn.evaluate(test_dataset)
print(f"Acurácia do modelo no conjunto de teste: {accuracy * 100:.2f}%")

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 426ms/step - accuracy: 0.5278 - loss: 1.2899
Acurácia do modelo no conjunto de teste: 48.53%


In [52]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def infer_image_directly(model, image_path, class_names):
    # Carregar e pré-processar a imagem
    img = load_img(image_path, target_size=(224, 224))  # Ajuste para o tamanho do modelo
    img_array = img_to_array(img) / 255.0  # Normalizar para [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Adicionar dimensão do batch

    # Fazer a predição
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=-1)  # Classe com maior probabilidade

    print(predictions)

    confidence = np.max(predictions)  # Confiança na predição

    print(f"Classe prevista: {class_names[predicted_class[0]]} ")
    return predicted_class[:]

# Classes do modelo
class_names = [
    "Rock",
    "Pop",
    "Jazz",
    "Eletronical",
    "Classical",
    "Country"
    ]

# Exemplo de inferência
image_path = '/content/espectrogramas/538.png'  # Caminho da imagem
infer_image_directly(cnn, image_path, class_names)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[0.2041483  0.05350558 0.32694575 0.15212919 0.06746759 0.19580351]]
Classe prevista: Jazz 


array([2])